In [21]:
# Begin with these LIMS exports and files placed in /input_inv:

# inv.xlsx - Agena total scanned inventory of XE numbers
# inv_lots.xlsx - reagent lots isolated
# lims_inv.csv - LIMS exported file of all reagents on shelf of interest

import pandas as pd
import numpy as np
import re

In [22]:
# Parse data from Inventory Workbook into single dataframe

scanned_inv = pd.concat(pd.read_excel(
    'input_inv/inv.xlsx',
    sheet_name=None
), ignore_index=True)

In [23]:
# Parse lot numbers from Lot Number Workbook into single .csv dataframe
parsed_lots = pd.concat(pd.read_excel(
    'input_inv/inv_lots.xlsx',
    sheet_name=None
), ignore_index=True)

In [24]:
#Trims full .csv export from LIMS to appropriate columns
lims_inv = pd.read_csv('input_inv/lims_inv.csv')
lims_inv = lims_inv[['Name', 'Barcode', 'Lot Number', 'Location']]

In [ ]:
#Selects for rows containing your inventory
#Input the title of your project, reagent name, or any keyword found in 'Name' of your project
reagents = ['Oligo'] 
lims_inv = lims_inv[
    lims_inv['Name'].str.contains('|'.join(reagents), case=False)]

In [28]:
#Regex function to strip location down to shelf.
lims_inv['Location'] = re.search('SH[\d]+', str(loc)).group(0)

In [38]:
#Creates list of scanned XE numbers from physical inventory as reference when comparing to LIMS inventory
scanned_xe_list = []
for xe in scanned_inv['XE Number']:
    xe = xe.upper()
    if xe[0] == 'X':
        scanned_xe_list.append(xe)

List created is equal to initial inventory


In [ ]:
#Checks for possible errors in your scanned inventory
if len(scanned_xe_list) == len(scanned_inv):
    print('List created is equal to initial inventory')
else:
    print('Your inventory may have formatting errors, check again',
          '\nLength of your inventory:',
          len(scanned_inv), '\nLength of created list:', len(scanned_xe_list))

In [ ]:
#Creates was_scanned column
#If LIMS reagent is physically present: True
#If reagent is missing or has not been trashed: False
lims_inv['Found in Inventory'] = ''
for xe in lims_inv['Barcode']:
    if xe in scanned_xe_list:
        lims_inv['Found in Inventory'].loc[lims_inv['Barcode'] == xe] = True
    else:
        lims_inv['Found in Inventory'].loc[lims_inv['Barcode'] == xe] = False

In [ ]:
#Sorts table so missing inventory is first, and then sorts by name alphabetically
lims_inv.sort_values(by=['Found in Inventory', 'Name'], inplace=True)

In [27]:
#Final scanned inventory .csv from .xlsx
scanned_inv.to_csv('output_inv/scanned_inv.csv')

#Final LIMS inventory .csv
lims_inv.to_csv('output_inv/lims_inv.csv')

#Lots of inventory isolated into .csv
parsed_lots.to_csv('output_inv/inventory_lots.csv')